In [1]:
import pandas as pd
df = pd.read_csv(r"C:\Users\mikem\Downloads\DATASETS\Customer_behavior\customer_shopping_behavior.csv")

In [2]:
df['Review Rating']= df.groupby('Category')['Review Rating'].transform(lambda x: x.fillna(x.median()))

In [3]:
df.isnull().sum()

Customer ID               0
Age                       0
Gender                    0
Item Purchased            0
Category                  0
Purchase Amount (USD)     0
Location                  0
Size                      0
Color                     0
Season                    0
Review Rating             0
Subscription Status       0
Shipping Type             0
Discount Applied          0
Promo Code Used           0
Previous Purchases        0
Payment Method            0
Frequency of Purchases    0
dtype: int64

In [4]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customer_id             3900 non-null   int64  
 1   age                     3900 non-null   int64  
 2   gender                  3900 non-null   object 
 3   item_purchased          3900 non-null   object 
 4   category                3900 non-null   object 
 5   purchase_amount_(usd)   3900 non-null   int64  
 6   location                3900 non-null   object 
 7   size                    3900 non-null   object 
 8   color                   3900 non-null   object 
 9   season                  3900 non-null   object 
 10  review_rating           3900 non-null   float64
 11  subscription_status     3900 non-null   object 
 12  shipping_type           3900 non-null   object 
 13  discount_applied        3900 non-null   object 
 14  promo_code_used         3900 non-null   

In [5]:
df = df.rename(columns = {'purchase_amount_(usd)': 'purchase_amount'})

In [6]:
#create a new column age_group
labels = ['Young Adult','Adult', 'Middle-aged', 'Senior']
df['age_group']= pd.qcut(df['age'], q=4, labels = labels)

In [7]:
#create column purchase_frequency_days
frequency_mapping = {'Fortnightly':14, 'Weekly':7, 'Monthly':30,'Quarterly':90,'Bi-Weekly':14, 'Annually':365, 'Every 3 Months':90}
df['purchase_frequency_days'] = df['frequency_of_purchases'].map(frequency_mapping)
df[['frequency_of_purchases','purchase_frequency_days']].head(10)

,frequency_of_purchases,purchase_frequency_days
0,Fortnightly,14
1,Fortnightly,14
2,Weekly,7
3,Weekly,7
4,Annually,365
5,Weekly,7
6,Quarterly,90
7,Weekly,7
8,Annually,365
9,Quarterly,90


In [8]:
(df['discount_applied'] == df['promo_code_used']).all()

np.True_

In [9]:
df = df.drop('promo_code_used', axis= 1)

In [10]:
pip install psycopg2-binary sqlalchemy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from sqlalchemy import create_engine
#connecting to postgresql
username = 'postgres'
password = '123456'
host = 'localhost'
port = '5432'
database = 'customer_behavior'
engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}")
#loading df into postgresql
table_name = "customer" 
df.to_sql(table_name, engine, if_exists="replace", index= False)

print(f"Data successfully loaded into table '{table_name}' in database '{database}'.")

Data successfully loaded into table 'customer' in database 'customer_behavior'.
